

# Adım 1: Gerekli Kütüphaneleri Yükleme ve Veriyi Excel Olarak Okuma



In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
import warnings
warnings.filterwarnings('ignore') # Uyarıları gizle

# 0. openpyxl kütüphanesini kurma (Excel dosyalarını okumak için zorunludur)
!pip install openpyxl

# 1. Google Drive'ı Bağlama
drive.mount('/content/drive')


file_path = '/content/drive/MyDrive/mobile_game_inapp_purchases.xlsx'

# Veriyi oku: pd.read_excel() fonksiyonunu kullanıyoruz.
try:
    # İlk sayfayı okuyoruz. Eğer sayfa adı farklıysa 'Sheet1' yerine o adı yazın.
    df = pd.read_excel(file_path, sheet_name='Sheet1')

    print("--- EXCEL Veri Yükleme Başarılı ---")
    print("Veri Setinin Özeti (İlk 5 Satır):")
    print(df.head())
    print("\nVeri Tipleri ve Eksik Değerler (df.info()):")
    df.info()

except FileNotFoundError:
    print(f"\nFATAL HATA: Dosya yolu bulunamadı. Lütfen dosya adının ve yolunun ('{file_path}') birebir aynı olduğundan emin olun.")
except Exception as e:
    print(f"\nBilinmeyen bir hata oluştu: {type(e).__name__} - {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- EXCEL Veri Yükleme Başarılı ---
Veri Setinin Özeti (İlk 5 Satır):
                                 UserID   Age  Gender      Country   Device  \
0  c9889ab0-9cfc-4a75-acd9-5eab1df0015c  49.0    Male       Norway  Android   
1  7c9e413c-ecca-45f2-a780-2826a07952a2  15.0    Male  Switzerland      iOS   
2  fd61e419-1a92-4f43-a8c7-135842ad328a  23.0    Male        China  Android   
3  bdb7f6d1-ff9a-468c-afe7-43f32a94293e  31.0    Male       Mexico  Android   
4  aa7eec14-4846-47b9-b879-9c98038cda04  37.0  Female        India  Android   

       GameGenre  SessionCount  AverageSessionLength SpendingSegment  \
0  Battle Royale             9                 12.83          Minnow   
1     Action RPG            11                 19.39          Minnow   
2       Fighting             9                  8.87          Minnow   
3         Racing            12        

#Adım 2: Veri Temizleme ve Tarih Formatlarını Düzeltme

In [ ]:
# 1. Monetization Sütunlarındaki Eksik Değer Yönetimi (Non-Payers)
# Bu, harcama yapmayan 136 kullanıcıyı doğru şekilde sınıflandırır.
df['InAppPurchaseAmount'] = df['InAppPurchaseAmount'].fillna(0.0)
df['PaymentMethod'] = df['PaymentMethod'].fillna('Non-Payer')
df['FirstPurchaseDaysAfterInstall'] = df['FirstPurchaseDaysAfterInstall'].fillna(999) # Harcama yapmayanlar için yüksek bir değer

# 2. Demografik Sütunlardaki Eksik Değer Yönetimi
# Yaş, Cinsiyet vb. 60 kullanıcının eksik verisini 'Bilinmeyen' olarak dolduruyoruz.
# Bu kullanıcıları analizin dışında bırakmak yerine, ayrı bir segment olarak analiz etme imkanı sunar.
for col in ['Age', 'Gender', 'Country', 'Device', 'GameGenre', 'SpendingSegment']:
    # Sayısal değerleri ortalama/medyan ile doldurmak yerine 'Bilinmeyen' kategorisi oluşturmak daha güvenli.
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('Bilinmeyen')
    elif df[col].dtype == 'float64':
         # Yaş verisini de 'Bilinmeyen' olarak doldurmak için object tipine dönüştürüp sonra dolduruyoruz
         df[col] = df[col].fillna(-1) # Yaşı -1 olarak işaretliyoruz.

# 3. LastPurchaseDate zaten doğru formatta, ek işlem gerekmiyor.

print("Temizlik aşaması tamamlandı. Tüm eksik değerler kontrolü:")
print(df.isnull().sum())

Temizlik aşaması tamamlandı. Tüm eksik değerler kontrolü:
UserID                             0
Age                                0
Gender                             0
Country                            0
Device                             0
GameGenre                          0
SessionCount                       0
AverageSessionLength               0
SpendingSegment                    0
InAppPurchaseAmount                0
FirstPurchaseDaysAfterInstall      0
PaymentMethod                      0
LastPurchaseDate                 136
dtype: int64


#Adım 3: Analitik Özellik Mühendisliği (Feature Engineering)

In [ ]:
# A. Ödeme Durumu Sütunu (IsPayer - Dönüşüm Oranı için)
df['IsPayer'] = np.where(df['InAppPurchaseAmount'] > 0, 1, 0)

# B. Yaş Grubu Sınıflandırması (AgeGroup) - Hata Düzeltildi
# Bins: 6 adet sınır
bins = [-2, 10, 25, 40, 55, 70]
# Labels: 5 adet etiket (6-1=5)
labels = ['Bilinmeyen/Genç (<10)', 'Genç (10-25)', 'Yetişkin (26-40)', 'Kıdemli (41-55)', 'Uzman (56+)']

# Yaş gruplaması (Age=-1 olanlar otomatik olarak 'Bilinmeyen/Genç' grubuna düşecektir)
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=True, include_lowest=True)

# C. Aktivite Seviyesi Sınıflandırması (ActivityLevel)
df['ActivityLevel'] = pd.cut(df['SessionCount'],
                             bins=[0, 5, 15, df['SessionCount'].max() + 1],
                             labels=['Düşük (<5)', 'Orta (5-15)', 'Yüksek (>15)'],
                             right=False)

# D. Son Satın Almadan Bu Yana Geçen Gün (DaysSinceLastPurchase) - Churn Analizi
current_date = pd.to_datetime('2025-12-16') # Sabit analiz tarihi (Bugün)
df['DaysSinceLastPurchase'] = (current_date - df['LastPurchaseDate']).dt.days
# LastPurchaseDate NaN olanlar için gün sayısını yüksek bir değerle işaretleyerek NaN'i kaldırıyoruz
df['DaysSinceLastPurchase'] = df['DaysSinceLastPurchase'].fillna(999)


print("Özellik Mühendisliği tamamlandı. Son veri setinin ilk 5 satırı ve yeni sütunlar:")
print(df[['UserID', 'InAppPurchaseAmount', 'IsPayer', 'AgeGroup', 'ActivityLevel', 'DaysSinceLastPurchase']].head())

Özellik Mühendisliği tamamlandı. Son veri setinin ilk 5 satırı ve yeni sütunlar:
                                 UserID  InAppPurchaseAmount  IsPayer  \
0  c9889ab0-9cfc-4a75-acd9-5eab1df0015c                11.40        1   
1  7c9e413c-ecca-45f2-a780-2826a07952a2                 6.37        1   
2  fd61e419-1a92-4f43-a8c7-135842ad328a                15.81        1   
3  bdb7f6d1-ff9a-468c-afe7-43f32a94293e                13.49        1   
4  aa7eec14-4846-47b9-b879-9c98038cda04                10.86        1   

           AgeGroup ActivityLevel  DaysSinceLastPurchase  
0   Kıdemli (41-55)   Orta (5-15)                  272.0  
1      Genç (10-25)   Orta (5-15)                  191.0  
2      Genç (10-25)   Orta (5-15)                  197.0  
3  Yetişkin (26-40)   Orta (5-15)                  259.0  
4  Yetişkin (26-40)   Orta (5-15)                  225.0  


#Adım 4: Temiz Veri Setini Kaydetme

In [ ]:
# Drive'da kaydedilecek dosya yolu
output_file_path = '/content/drive/MyDrive/clean_game_data_for_powerbi.csv'

# Veri setini kaydetme (index'i hariç tutarak)
df.to_csv(output_file_path, index=False)

print(f"\n--- PYTHON AŞAMASI BAŞARIYLA TAMAMLANDI ---")
print(f"Temizlenmiş veri seti başarıyla kaydedildi: {output_file_path}")
print("Power BI ile modelleme ve raporlama adımlarına geçmeye hazırsınız.")


--- PYTHON AŞAMASI BAŞARIYLA TAMAMLANDI ---
Temizlenmiş veri seti başarıyla kaydedildi: /content/drive/MyDrive/clean_game_data_for_powerbi.csv
Power BI ile modelleme ve raporlama adımlarına geçmeye hazırsınız.
